In [40]:
import pandas as pd
import numpy as np
import json

#reviews_datasets = pd.read_json('influencer_score_covid19 AND government AND lockdown.json')
reviews_datasets = pd.read_csv('C:/Users/Krish/Desktop/final.csv')
reviews_datasets = reviews_datasets.head(10000)
#print(reviews_datasets)
reviews_datasets.dropna()
reviews_datasets.head()

,created_at,full_text,id,user,entities,retweet_count,favorite_count,country,influencer_score
0,2020-09-18T15:00:00Z,"איחולים לבביים לידידי @netanyahu נתניהו, לאזרח...",1306969908115394561,"{'id': 18839785, 'id_str': '18839785', 'name':...","{'hashtags': [], 'symbols': [], 'user_mentions...",2157,20115,India,22272
1,2020-09-18T15:00:00Z,Warm Rosh Hashanah greetings to my friend @net...,1306969864217690112,"{'id': 18839785, 'id_str': '18839785', 'name':...","{'hashtags': [], 'symbols': [], 'user_mentions...",2441,20117,India,22558
2,2020-09-18T11:00:00Z,मैं देश के किसानों को स्पष्ट संदेश देना चाहता ...,1306915656600702976,"{'id': 18839785, 'id_str': '18839785', 'name':...","{'hashtags': [], 'symbols': [], 'user_mentions...",10668,43452,India,54120
3,2020-09-18T11:00:00Z,देशभर के किसानों को कृषि सुधार विधेयकों के पार...,1306915613101535232,"{'id': 18839785, 'id_str': '18839785', 'name':...","{'hashtags': [], 'symbols': [], 'user_mentions...",4327,17658,India,21985
4,2020-09-18T11:00:00Z,आज हाजीपुर-घोसवर-वैशाली नई रेल लाइन के शुरू हो...,1306915261681790978,"{'id': 18839785, 'id_str': '18839785', 'name':...","{'hashtags': [], 'symbols': [], 'user_mentions...",1828,7947,India,9775


In [9]:
import nltk
from nltk.corpus import stopwords
stop_words = []
for x in stopwords.words('english'):
    stop_words.append(x)
for y in stopwords.words('italian'):
    stop_words.append(y)
for z in stopwords.words('hindi'):
    stop_words.append(z)
my_stopwords = frozenset(stop_words)
#print(stopwords.words('italian'))

In [10]:
reviews_datasets['full_text'][100]

'Thanks for your touching wishes. https://t.co/7pUajtQM9P'

In [41]:
poi_tweets = []
import json
import ast
for i in range(len(reviews_datasets)):
    x = reviews_datasets['user'][i]
    y = json.loads(json.dumps(ast.literal_eval(x)))
    #if(y['verified']):
    poi_tweets.append( (reviews_datasets['full_text'][i]).replace('SCREEN_NAME',' '))

In [46]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words=my_stopwords)
doc_term_matrix = count_vect.fit_transform(poi_tweets)


C:\Users\krish\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['अपन', 'अभ', 'आद', 'इत', 'इन', 'इनक', 'इसक', 'इसम', 'उनक', 'उसक', 'एव', 'ऐस', 'करत', 'करन', 'कह', 'कहत', 'गय', 'जह', 'तन', 'तर', 'दब', 'दर', 'धर', 'नक', 'नस', 'नह', 'पहल', 'बन', 'बह', 'यत', 'यद', 'रख', 'रह', 'लक', 'वर', 'वग़', 'सकत', 'सबस', 'सभ', 'सर'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [47]:
doc_term_matrix

<1173x2210 sparse matrix of type '<class 'numpy.int64'>'
	with 16217 stored elements in Compressed Sparse Row format>

In [48]:
from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(doc_term_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [49]:
import random

for i in range(10):
    random_id = random.randint(0,len(count_vect.get_feature_names()))
    print(count_vect.get_feature_names()[random_id])

રક
virtual
post
issued
st
addressing
death
रमण
abundance
शक


In [50]:
first_topic = LDA.components_[0]

In [51]:
top_topic_words = first_topic.argsort()[-10:]

In [52]:
for i in top_topic_words:
    print(count_vect.get_feature_names()[i])

india
life
rt
covid
ji
may
covid19
government
https
co


In [55]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['india', 'life', 'rt', 'covid', 'ji', 'may', 'covid19', 'government', 'https', 'co']


Top 10 words for topic #1:
['ji', 'people', 'thanks', '19', 'covid', 'india', 'wishes', 'thank', 'https', 'co']


Top 10 words for topic #2:
['आभ', 'well', 'nation', 'यव', 'धन', 'आपक', 'बह', 'india', 'https', 'co']


Top 10 words for topic #3:
['कल', 'अपन', 'मन', 'सरक', 'जन', 'रह', 'हम', 'रत', 'https', 'co']


Top 10 words for topic #4:
['shri', 'condolences', 'people', 'ji', 'sector', 'family', 'would', 'education', 'co', 'https']




In [56]:
topic_values = LDA.transform(doc_term_matrix)
topic_values.shape

(1173, 5)

In [57]:
reviews_datasets['Topic'] = topic_values.argmax(axis=1)

ValueError: Length of values does not match length of index

In [58]:
reviews_datasets['Topic'].head()

0    3
1    2
2    0
3    0
4    0
Name: Topic, dtype: int64

In [24]:
from langdetect import detect
import spacy
spacy.load('en')
spacy.load('hi')
spacy.load('it')
from spacy.lang.en import English
from spacy.lang.hi import Hindi
from spacy.lang.hi import Italian
english_parser = English()

OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [38]:
import requests

url = ('http://newsapi.org/v2/everything?'
       'q=India&'
       'from=2020-11-12&'
       'sortBy=popularity&'
       'apiKey=8ab0444ae6e74d0eb01c52f941647514')

response = requests.get(url)

print(json.dumps(response.json()['articles']))

[{"source": {"id": null, "name": "Lifehacker.com"}, "author": "Aisha Jordan", "title": "How to Celebrate Diwali", "description": "Diwali, a five-day-long \u201cfestival of lights\u201d that celebrates the triumph of good over evil, began yesterday. The holiday emerged from the Hindu religion, but has become a cultural event around the world. We could all use a little celebration of triumph over e\u2026", "url": "https://lifehacker.com/how-to-celebrate-diwali-1845666878", "urlToImage": "https://i.kinja-img.com/gawker-media/image/upload/c_fill,f_auto,fl_progressive,g_center,h_675,pg_1,q_80,w_1200/zrr7tme5jm9sf7ofjqhc.jpg", "publishedAt": "2020-11-13T21:15:00Z", "content": "Diwali, a five-day-long festival of lights that celebrates the triumph of good over evil, began yesterday. The holiday emerged from the Hindu religion, but has become a cultural event around the worl\u2026 [+1951 chars]"}, {"source": {"id": "engadget", "name": "Engadget"}, "author": "Daniel Cooper", "title": "Google Ma

In [42]:
import spacy
spacy.load('en_core_web_sm')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [43]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\krish\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [60]:
nltk.download('stopwords')
en_stop = list(nltk.corpus.stopwords.words('english'))
en_stop.append('SCREEN_NAME')
en_stop = set(en_stop)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\krish\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [61]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [62]:
import random
text_data = []
for line in poi_tweets:
    tokens = prepare_text_for_lda(line)
    if random.random() > .99:
        try:
            text_data.append(tokens)
        except:
            text_data.append(tokens)

In [63]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [64]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.026*"सम्मान" + 0.015*"unemployment" + 0.014*"covid19" + 0.014*"government"')
(1, '0.029*"coronavirus" + 0.026*"सरकार" + 0.023*"covid19" + 0.023*"कोरोना"')
(2, '0.099*"covid19" + 0.096*"dangerous" + 0.096*"pandemic" + 0.095*"unemployment"')
(3, '0.036*"सरकार" + 0.029*"lockd" + 0.029*"कितने" + 0.029*"आंकड़े"')
(4, '0.075*"covid19" + 0.059*"people" + 0.055*"effort" + 0.055*"concern"')


In [65]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)
ldamodel.save('model10.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.116*"17baje17minute" + 0.099*"unemployment" + 0.099*"dangerous" + 0.096*"covid19"')
(1, '0.026*"covid19" + 0.026*"india" + 0.026*"prime" + 0.026*"minister"')
(2, '0.037*"government" + 0.025*"सरकार" + 0.025*"number" + 0.025*"case"')
(3, '0.039*"कोरोना" + 0.026*"सरकार" + 0.026*"संक्रमण" + 0.014*"coronavirus"')
(4, '0.042*"fail" + 0.022*"pandemic" + 0.022*"coronavirus" + 0.022*"incompetence"')
(5, '0.030*"सरकार" + 0.030*"covid19" + 0.016*"government" + 0.016*"ग्रामीण"')
(6, '0.031*"covid19" + 0.031*"change" + 0.031*"pandemic" + 0.016*"effort"')
(7, '0.096*"covid19" + 0.089*"pandemic" + 0.089*"unemployment" + 0.089*"youngster"')
(8, '0.102*"covid19" + 0.079*"people" + 0.074*"effort" + 0.074*"concern"')
(9, '0.056*"सरकार" + 0.029*"सम्मान" + 0.029*"मंत्री" + 0.029*"कितने"')


In [67]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model10.gensim')
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)
pyLDAvis.save_html(lda_display, 'lda.html')